In [ ]:
import kagglehub
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer
from torch.nn import functional as F
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
from transformers import T5Tokenizer

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
path = kagglehub.dataset_download("gowrishankarp/newspaper-text-summarization-cnn-dailymail")
print("Path to dataset files:", path)

100%|██████████| 503M/503M [00:23<00:00, 22.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/gowrishankarp/newspaper-text-summarization-cnn-dailymail/versions/2


In [ ]:
train_file = os.path.join(path, 'cnn_dailymail', 'train.csv')
val_file = os.path.join(path, 'cnn_dailymail', 'validation.csv')
test_file = os.path.join(path, 'cnn_dailymail', 'test.csv')
df_train = pd.read_csv(train_file).drop(columns=['id'])
df_val = pd.read_csv(val_file).drop(columns=['id'])
df_test = pd.read_csv(test_file).drop(columns=['id'])
X_traindata, y_traindata = df_train['article'], df_train['highlights']
X_valdata, y_valdata = df_val['article'], df_val['highlights']
X_testdata, y_testdata = df_test['article'], df_test['highlights']

In [ ]:
X_traindata, y_traindata = df_train['article'].head(5000), df_train['highlights'].head(5000)
# X_valdata, y_valdata = df_val['article'].head(10000), df_val['highlights'].head(10000)
# X_testdata, y_testdata = df_test['article'].head(100), df_test['highlights'].head(100)


In [ ]:

# class SummaryDataset(Dataset):
#     def __init__(self, articles, highlights, tokenizer, max_length):
#         self.articles = articles
#         self.highlights = highlights
#         self.tokenizer = tokenizer
#         self.max_length = max_length

#     def __len__(self):
#         return len(self.articles)

#     def __getitem__(self, idx):
#         article = str(self.articles[idx])
#         highlight = str(self.highlights[idx])

#         # Tokenize inputs and labels
#         article_encodings = self.tokenizer(
#             article, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt"
#         )
#         highlight_encodings = self.tokenizer(
#             highlight, max_length=self.max_length, padding='max_length', truncation=True, return_tensors="pt"
#         )

#         return (
#             article_encodings['input_ids'].squeeze(0),  # 0 for input_ids of article
#             highlight_encodings['input_ids'].squeeze(0)  # 1 for input_ids of highlight/summary
#         )

class SummaryDataset(Dataset):
    def __init__(self, articles, highlights, tokenizer, encoder_maxlen, decoder_maxlen):
        self.articles = articles
        self.highlights = highlights
        self.tokenizer = tokenizer
        self.encoder_maxlen = encoder_maxlen  # max length for the articles
        self.decoder_maxlen = decoder_maxlen  # max length for the highlights/summary

    def __len__(self):
        return len(self.articles)

    def __getitem__(self, idx):
        article = str(self.articles[idx])
        highlight = str(self.highlights[idx])

        # Tokenize articles (inputs) with encoder_maxlen
        article_encodings = self.tokenizer(
            article,
            max_length=self.encoder_maxlen,  # Use encoder_maxlen for articles
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        # Tokenize highlights (targets) with decoder_maxlen
        highlight_encodings = self.tokenizer(
            highlight,
            max_length=self.decoder_maxlen,  # Use decoder_maxlen for highlights/summary
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        return (
            article_encodings['input_ids'].squeeze(0),  # 0 for input_ids of article
            highlight_encodings['input_ids'].squeeze(0)  # 1 for input_ids of highlight/summary
        )


# Initialize Longformer tokenizer and datase

# Initialize T5 tokenizer and dataset
# tokenizer = T5Tokenizer.from_pretrained('t5-small')  # Use T5 tokenizer for summarization
# max_length = 512

# Assuming X_traindata, y_traindata, etc. are your datasets
# Define your encoder_maxlen and decoder_maxlen
decoder_maxlen = max(len(tokenizer.encode(summ, truncation=True)) for summ in y_traindata)

encoder_maxlen = 400  # Max length for articles
# Initialize T5 tokenizer and dataset
tokenizer = T5Tokenizer.from_pretrained('t5-large')  # Use T5 tokenizer for summarization
max_length = 768

# Assuming X_traindata, y_traindata, etc. are your datasets
train_dataset = SummaryDataset(X_traindata, y_traindata, tokenizer, encoder_maxlen, decoder_maxlen)
val_dataset = SummaryDataset(X_valdata, y_valdata, tokenizer, encoder_maxlen, decoder_maxlen)
test_dataset = SummaryDataset(X_testdata, y_testdata, tokenizer, encoder_maxlen, decoder_maxlen)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

In [ ]:
# from torch.utils.data import Dataset
# from transformers import T5Tokenizer

# class SummaryDataset(Dataset):
#     def __init__(self, articles, summaries, tokenizer, max_length=512, stride=256):
#         """
#         SummaryDataset initializes with article text, target summaries, and tokenizer.

#         Args:
#             articles (list): List of article texts.
#             summaries (list): List of summary texts corresponding to the articles.
#             tokenizer (T5Tokenizer): Tokenizer for encoding articles and summaries.
#             max_length (int): Maximum token length for each sequence (default 512).
#             stride (int): Step size to slide over the article if it exceeds max_length (default 256).
#         """
#         self.articles = articles
#         self.summaries = summaries
#         self.tokenizer = tokenizer
#         self.max_length = max_length
#         self.stride = stride

#     def __len__(self):
#         return len(self.articles)

#     def __getitem__(self, idx):
#         article = self.articles[idx]
#         summary = self.summaries[idx]

#         # Encode the article with sliding window if it exceeds max_length
#         article_inputs = self.tokenizer(
#             article,
#             max_length=self.max_length,
#             padding="max_length",
#             truncation=True,
#             return_tensors="pt",
#             stride=self.stride,
#             return_overflowing_tokens=True
#         )

#         summary_inputs = self.tokenizer(
#             summary,
#             max_length=self.max_length,
#             padding="max_length",
#             truncation=True,
#             return_tensors="pt"
#         )

#         # Return the input and target IDs, each possibly containing multiple splits
#         input_ids = article_inputs["input_ids"].squeeze(0)  # (num_chunks, max_length)
#         target_ids = summary_inputs["input_ids"].squeeze(0)

#         return input_ids, target_ids
# # Initialize T5 tokenizer and dataset
# tokenizer = T5Tokenizer.from_pretrained('t5-small')  # Use T5 tokenizer for summarization
# max_length = 512

# # Assuming X_traindata, y_traindata, etc. are your datasets
# train_dataset = SummaryDataset(X_traindata, y_traindata, tokenizer, max_length)
# val_dataset = SummaryDataset(X_valdata, y_valdata, tokenizer, max_length)
# test_dataset = SummaryDataset(X_testdata, y_testdata, tokenizer, max_length)

In [ ]:
def angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return pos * angle_rates
def positional_encoding(position, d_model):
    angle_rads = angles(np.arange(position)[:, np.newaxis],
                        np.arange(d_model)[np.newaxis, :],
                        d_model)
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return torch.from_numpy(pos_encoding)

In [ ]:
# def padding_mask(seq):
#   seq = torch.eq(seq, 0).float()
#   return seq[:, np.newaxis, np.newaxis, :]
# def look_ahead_mask(size):
#   mask = torch.triu(torch.ones((1, size, size)), diagonal=1)
#   return mask == 0

In [ ]:
def create_padding_mask(seq):
    mask = (seq == 0).float()
    return mask[:, None, None, :]

def create_look_ahead_mask(size):
    mask = torch.triu(torch.ones((size, size)), diagonal=1)
    return mask

In [ ]:
# def create_padding_mask(seq):
#   mask = (seq == 0).type(torch.bool)
#   return mask[:, None, None, :]
# def create_look_ahead_mask(size):
#   mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.bool)
#   return mask

In [ ]:
def scaled_dot_product_attention(q, k, v, mask=None):
  q, k, v = q.to(device), k.to(device), v.to(device)
  depth = q.size(-1)
  dots = torch.matmul(q, k.transpose(-2, -1))

  if mask is not None:
      dots = dots.masked_fill(mask.bool(), -1e9)
  attention_weights = F.softmax(dots, dim=-1)
  output = torch.matmul(attention_weights, v)

  return output, attention_weights
def pointwise_ffn(d_model, dff):
  return torch.nn.Sequential(
      torch.nn.Linear(d_model, dff),
      torch.nn.ReLU(),
      torch.nn.Linear(dff, d_model)
  ).to(device)

In [ ]:
class MultiHeadAttention(nn.Module):
  def __init__(self, d_model, num_heads):
    super(MultiHeadAttention, self).__init__()
    self.num_heads = num_heads
    self.d_model = d_model

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads
    #Linear layer for q k v
    self.wq = nn.Linear(d_model, d_model).to(device)
    self.wk = nn.Linear(d_model, d_model).to(device)
    self.wv = nn.Linear(d_model, d_model).to(device)
    # Final linear layer
    self.dense = nn.Linear(d_model, d_model).to(device)
  def split_heads(self, x, batch_size):  # Added split_heads function
        x = x.view(batch_size, -1, self.num_heads, self.depth)
        return x.permute(0, 2, 1, 3)
  def forward(self, v, k, q, mask=None):
        batch_size = q.size(0)
        q, k, v = q.to(device), k.to(device), v.to(device)
        # Apply linear transformations to queries, keys, and values
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        # Split the heads for multi-head attention
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        # Compute scaled dot-product attention
        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask)
        # Concatenate the attention outputs
        scaled_attention = scaled_attention.permute(0, 2, 1, 3)
        concat_attention = scaled_attention.contiguous().view(batch_size, -1, self.d_model)

        # Pass through the final dense layer
        output = self.dense(concat_attention)

        return output, attention_weights

In [ ]:
class EncoderLayer(nn.Module):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(EncoderLayer, self).__init__()
    self.mha = MultiHeadAttention(d_model, num_heads)
    self.ffn = pointwise_ffn(d_model, dff)

    self.layernorm1 = nn.LayerNorm(d_model, eps=1e-6).to(device)
    self.layernorm2 = nn.LayerNorm(d_model, eps=1e-6).to(device)

    self.dropout1 = nn.Dropout(rate).to(device)
    self.dropout2 = nn.Dropout(rate).to(device)

  def forward(self, x, training, mask):
    x = x.to(device)
    attention_output, _ = self.mha(x, x, x, mask)
    attention_output = self.dropout1(attention_output)
    out1 = self.layernorm1(x + attention_output)

    ffn_output = self.ffn(out1)
    ffn_output = self.dropout2(ffn_output)
    out2 = self.layernorm2(out1 + ffn_output)

    return out2

In [ ]:
class DecoderLayer(nn.Module):
  def __init__(self, d_model, num_heads, dff, rate=0.1):
    super(DecoderLayer, self).__init__()

    self.mha1 = MultiHeadAttention(d_model, num_heads)
    self.mha2 = MultiHeadAttention(d_model, num_heads)

    self.fnn = pointwise_ffn(d_model, dff)

    self.layernorm1 = nn.LayerNorm(d_model, eps=1e-6).to(device)
    self.layernorm2 = nn.LayerNorm(d_model, eps=1e-6).to(device)
    self.layernorm3 = nn.LayerNorm(d_model, eps=1e-6).to(device)

    self.dropout1 = nn.Dropout(rate).to(device)
    self.dropout2 = nn.Dropout(rate).to(device)
    self.dropout3 = nn.Dropout(rate).to(device)

  def forward(self, x, enc_output, training, look_ahead_mask, padding_mask):
    x = x.to(device)
    enc_output = enc_output.to(device)
    att1, att_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
    att1 = self.dropout1(att1)
    out1 = self.layernorm1(att1 + x)

    att2, att_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
    att2 = self.dropout2(att2)
    out2 = self.layernorm2(att2 + out1)

    ffn_output = self.fnn(out2)
    ffn_output = self.dropout3(ffn_output)
    out3 = self.layernorm3(ffn_output + out2)

    return out3, att_weights_block1, att_weights_block2

In [ ]:
class Encoder(nn.Module):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
    super(Encoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = nn.Embedding(input_vocab_size, d_model).to(device)
    self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model).to(device)
    self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

    self.dropout = nn.Dropout(rate).to(device)

  def forward(self, x, training, mask):
    x = x.to(device)
    seq_len = x.shape[1]
    x = self.embedding(x)
    x *= torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32)) #d=d
    x += self.pos_encoding[:, :seq_len, :]
    x = self.dropout(x)
    for i in range(self.num_layers):
      x = self.enc_layers[i](x, training, mask)
    return x

In [ ]:
class Decoder(nn.Module):
  def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
    super(Decoder, self).__init__()
    self.d_model = d_model
    self.num_layers = num_layers

    self.embedding = nn.Embedding(target_vocab_size, d_model).to(device)
    self.pos_encoding = positional_encoding(maximum_position_encoding, d_model).to(device)
    self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
    self.dropout = nn.Dropout(rate).to(device)

  def forward(self, x, enc_output, training, look_ahead_mask, padding_mask):
    x = x.to(device)
    enc_output = enc_output.to(device)
    seq_len = x.shape[1]
    attention_weights = {}
    #GPT

    x = self.embedding(x)
    x *= torch.sqrt(torch.tensor(self.d_model, dtype=torch.float32))
    x += self.pos_encoding[:, :seq_len, :]
    x = self.dropout(x)

    for i in range(self.num_layers):
      x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)
      attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
      attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    return x, attention_weights

In [ ]:
class Transformer(nn.Module):
  def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
    super(Transformer, self).__init__()
    self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

    self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)
    self.final_layer = nn.Linear(d_model, target_vocab_size).to(device)

  def forward(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
    inp = inp.to(device)
    tar = tar.to(device)
    enc_padding_mask = enc_padding_mask.to(device)
    look_ahead_mask = look_ahead_mask.to(device)
    dec_padding_mask = dec_padding_mask.to(device)
    enc_output = self.encoder(inp, training, enc_padding_mask)

    dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

    final_output = self.final_layer(dec_output)

    return final_output, attention_weights

In [ ]:
transformer = Transformer(
    num_layers=4,
    d_model=128,
    num_heads=8,
    dff=512,
    input_vocab_size=tokenizer.vocab_size,
    target_vocab_size=tokenizer.vocab_size,
    pe_input=768,
    pe_target=768,
).to(device)

In [ ]:
from torch.optim import Adam
from transformers import get_linear_schedule_with_warmup

optimizer = Adam(transformer.parameters(), lr=0.01)
loss = nn.CrossEntropyLoss()


def loss_func(real, pred):
  """
  Calculates the cross-entropy loss between the predicted and real sequences.

  Args:
    real: The ground truth target sequence tensor.
    pred: The predicted sequence tensor.

  Returns:
    The average cross-entropy loss.
  """
  # real shape: (batch_size, seq_len)
  # pred shape: (batch_size, seq_len, vocab_size)

  # Reshape tensors for CrossEntropyLoss
  real = real.view(-1)  # (batch_size * seq_len)
  pred = pred.view(-1, pred.size(-1))  # (batch_size * seq_len, vocab_size)

  # Calculate and return the loss
  loss_ = F.cross_entropy(pred, real, ignore_index=0) # Ignore padding tokens (usually represented by 0)
  return loss_

In [ ]:
from torch.utils.data import DataLoader
import time

# Function to train the model
def train(model, train_dataset, val_dataset, epochs, batch_size, optimizer, loss_func):
    # Data loaders for training and validation
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    model = model.to(device)
    for epoch in range(epochs):
        model.train()  # Set model to training mode
        start_time = time.time()
        total_train_loss = 0

        for batch, (input_ids, target_ids) in enumerate(train_loader):
            # Move data to the device
            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)
            # Create masks
            enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(input_ids, target_ids)

            # Forward pass
            optimizer.zero_grad()
            predictions, _ = model(input_ids, target_ids, True, enc_padding_mask, look_ahead_mask, dec_padding_mask)

            # Calculate loss and backpropagate
            loss = loss_func(target_ids, predictions)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)
        val_loss = evaluate(model, val_loader, loss_func)
        time_elapsed = time.time() - start_time

        print(f"Epoch {epoch + 1}/{epochs} - "
              f"Train Loss: {avg_train_loss:.4f} - "
              f"Validation Loss: {val_loss:.4f} - "
              f"Time: {time_elapsed:.2f}s")

# Function to evaluate the model
def evaluate(model, val_loader, loss_func):
    model.eval()  # Set model to evaluation mode
    total_val_loss = 0

    with torch.no_grad():
        for input_ids, target_ids in val_loader:
            # Move data to the device
            input_ids = input_ids.to(device)
            target_ids = target_ids.to(device)
            # Create masks
            enc_padding_mask, look_ahead_mask, dec_padding_mask = create_masks(input_ids, target_ids)

            # Forward pass
            predictions, _ = model(input_ids, target_ids, False, enc_padding_mask, look_ahead_mask, dec_padding_mask)

            # Calculate validation loss
            loss = loss_func(target_ids, predictions)
            total_val_loss += loss.item()

    avg_val_loss = total_val_loss / len(val_loader)
    return avg_val_loss

# Function to create masks
# def create_masks(input_ids, target_ids):
#     input_ids = input_ids.to(device)
#     target_ids = target_ids.to(device)
#     enc_padding_mask = create_padding_mask(input_ids).to(device)
#     look_ahead_mask = create_look_ahead_mask(target_ids.size(1)).to(device)
#     dec_padding_mask = create_padding_mask(input_ids).to(device)
#     return enc_padding_mask, look_ahead_mask, dec_padding_mask
def create_masks(inp, tar):
    inp = inp.to(device)
    tar = tar.to(device)

    enc_padding_mask = create_padding_mask(inp).to(device)
    dec_padding_mask = create_padding_mask(inp).to(device)
    look_ahead_mask = create_look_ahead_mask(tar.shape[1]).to(device)
    dec_target_padding_mask = create_padding_mask(tar).to(device)
    combined_mask = torch.max(dec_target_padding_mask, look_ahead_mask).to(device)

    return enc_padding_mask, combined_mask, dec_padding_mask


In [ ]:
# Command to start training and display process
transformer.to(device)
epochs = 10
batch_size = 32
train(transformer, train_dataset, val_dataset, epochs, batch_size, optimizer, loss_func)


Epoch 1/10 - Train Loss: 1.4817 - Validation Loss: 0.2650 - Time: 259.80s
Epoch 2/10 - Train Loss: 0.0888 - Validation Loss: 0.1535 - Time: 266.34s
Epoch 3/10 - Train Loss: 0.0072 - Validation Loss: 0.1493 - Time: 270.76s
Epoch 4/10 - Train Loss: 0.0029 - Validation Loss: 0.1493 - Time: 263.52s
Epoch 5/10 - Train Loss: 0.0019 - Validation Loss: 0.1497 - Time: 261.57s
Epoch 6/10 - Train Loss: 0.0014 - Validation Loss: 0.1501 - Time: 262.70s
Epoch 7/10 - Train Loss: 0.0010 - Validation Loss: 0.1504 - Time: 261.93s
Epoch 8/10 - Train Loss: 0.0008 - Validation Loss: 0.1508 - Time: 262.96s
Epoch 9/10 - Train Loss: 0.0007 - Validation Loss: 0.1512 - Time: 262.78s
Epoch 10/10 - Train Loss: 0.0006 - Validation Loss: 0.1515 - Time: 261.83s


In [ ]:
# checkpoint_path = "checkpoint.pth"
# checkpoint = torch.load(checkpoint_path,  map_location=torch.device('cpu'))

# # Load the model state
# transformer.load_state_dict(checkpoint['model_state_dict'])
# transformer.eval()

In [ ]:
def generate_summary(input_text, tokenizer, model, max_length=512):
    """
    Generates a summary for a given input article using the trained transformer model.

    Args:
        input_text (str): The article text to summarize.
        tokenizer (T5Tokenizer): The tokenizer used to encode the input text and decode the output summary.
        model (Transformer): The trained transformer model.
        max_length (int): The maximum length of the input/output sequence for the model.

    Returns:
        str: The generated summary.
    """
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=max_length, padding="max_length", truncation=True)

    # Get the input_ids from the tokenized input
    input_ids = inputs["input_ids"].to(device)

    # Create padding mask for input
    enc_padding_mask = create_padding_mask(input_ids).to(device)

    # We don't need target sequence for generation, so start with a dummy token as the target
    target_ids = torch.zeros((input_ids.shape[0], max_length), dtype=torch.long).to(device)
    dec_padding_mask = create_padding_mask(target_ids).to(device)
    look_ahead_mask = create_look_ahead_mask(target_ids.size(1)).to(device)

    # Run the model to generate the output (summary)
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        # Get model predictions
        output, _ = model(input_ids, target_ids, False, enc_padding_mask, look_ahead_mask, dec_padding_mask)

    # Extract the generated summary from the output
    generated_ids = output.argmax(dim=-1)  # Get the predicted ids (index of max logits)

    # Decode the generated ids back into a text summary
    summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    return summary


In [ ]:
article = '''
Court documents released by investigators do not specify the name of the drug trafficking organization with which Mata allegedly conspired but says the organization has been importing narcotics from places such as Ecuador and the Dominican Republic by hiding them “inside shipping containers containing pallets of produce, including bananas.”

The organization “has been distributing narcotics in New Jersey and elsewhere,” the complaint says.

Authorities arrested Mata on Tuesday in Miami Gardens, Florida.

It was not immediately clear whether Mata has an attorney, and police officials could not be immediately reached for comment.
 '''
summary = generate_summary(article, tokenizer, transformer)
print("Generated Summary:", summary)

Generated Summary: travel travel travel  chief chiefcontamin travel        travel  Mc  travel  excess $1 chief Jackson chief Derby Mc    least excess  Saint  Derby least     least   Mc travel    travel  Derby   Pair Oil  least            Mc  excess $1 Derby experts least Derby Derby $1 least chief excess experts    travel  Derby least  excess $1    excess Derbydes  travel excess $1   $1   Mc $1 chief  Jackson   Mc chief least least  leastdes travelcontamin Jackson  Oil   travel  chief Comm excess travel des travel travel Cha  Ad least   $1  least Derby   least  chief excess  chief least Mc     Derby  excess $1 $1 $1         Jackson Ad travel Derby   Derby Mc     chief  Ad   travel chief   contamin travel least contamin       Comm    chiefcontamindes excess  Jackson  least  Mc  travel  travel   travel     Mc least  chief chief  experts chief  excess   travel $1 he Derby   travel $1   chief excess excess  $1 Ad Jackson excess Mc Jackson   Derby  Derby travel    least Jackson travel trave

In [ ]:
# def generate_summary(transformer, tokenizer, input_text, max_len=50, stop_word="<EOS>"):
#     """
#     Generates a summary for the given input text using the trained Transformer model.

#     Args:
#         transformer (nn.Module): The trained Transformer model.
#         tokenizer (T5Tokenizer): Tokenizer for encoding and decoding text.
#         input_text (str): The input article text to summarize.
#         max_len (int): Maximum length of the generated summary.
#         stop_word (str): The token indicating the end of the summary.

#     Returns:
#         str: The generated summary.
#     """
#     transformer.eval()  # Set the model to evaluation mode

#     # Tokenize the input text
#     input_ids = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=512).to(device)
#     enc_padding_mask = create_padding_mask(input_ids).to(device)

#     # Initialize the decoder input with the start token
#     decoder_input = torch.tensor([[tokenizer.pad_token_id]], device=device)

#     # Iteratively predict one word at a time
#     for _ in range(max_len):
#         # Create the look-ahead mask
#         look_ahead_mask = create_look_ahead_mask(decoder_input.size(1)).to(device)

#         # Forward pass through the model
#         predictions, _ = transformer(input_ids, decoder_input, False, enc_padding_mask, look_ahead_mask, enc_padding_mask)

#         # Select the last token from the predictions
#         last_token_logits = predictions[:, -1, :]
#         predicted_id = torch.argmax(last_token_logits, dim=-1)

#         # Append the predicted token to the decoder input
#         decoder_input = torch.cat([decoder_input, predicted_id.unsqueeze(0)], dim=1)

#         # Stop if the stop word is generated
#         if tokenizer.decode(predicted_id.item()) == stop_word:
#             break

#     # Decode the generated token IDs into text
#     summary = tokenizer.decode(decoder_input.squeeze().tolist(), skip_special_tokens=True)
#     return summary


In [ ]:
# # Example input text
# input_text = '''
# Court documents released by investigators do not specify the name of the drug trafficking organization with which Mata allegedly conspired but says the organization has been importing narcotics from places such as Ecuador and the Dominican Republic by hiding them “inside shipping containers containing pallets of produce, including bananas.”

# The organization “has been distributing narcotics in New Jersey and elsewhere,” the complaint says.

# Authorities arrested Mata on Tuesday in Miami Gardens, Florida.

# It was not immediately clear whether Mata has an attorney, and police officials could not be immediately reached for comment.
#     '''

# # Generate the summary
# summary = generate_summary(transformer, tokenizer, input_text)
# print("Summary:", summary)


Summary: least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least least


In [ ]:
# def evaluate(model, input_text, tokenizer, max_length=512, decoder_maxlen=100):
#     """Generates summaries from input text using the trained Transformer model.

#     Args:
#         model: The trained Transformer model.
#         input_text: The input text string.
#         tokenizer: The T5Tokenizer used for encoding.
#         max_length: Maximum token length for encoding the input.
#         decoder_maxlen: Maximum length for the generated summary.

#     Returns:
#         The generated summary string.
#     """
#     model.eval()
#     input_ids = tokenizer(
#         input_text, max_length=max_length, padding='max_length', truncation=True, return_tensors="pt"
#     )['input_ids'].to(device)
#     encoder_input = input_ids

#     decoder_input = torch.tensor([tokenizer.pad_token_id]).unsqueeze(0).to(device)  # Start with a PAD token
#     output = decoder_input

#     generated_ids = [] # Initialize an empty list to store generated IDs

#     for i in range(decoder_maxlen):
#         enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

#         with torch.no_grad():
#             predictions, _ = model(
#                 encoder_input,
#                 output,
#                 False,
#                 enc_padding_mask,
#                 combined_mask,
#                 dec_padding_mask
#             )
#         predictions = predictions[:, -1:, :]
#         predicted_id = torch.argmax(predictions, dim=-1)

#         if predicted_id == tokenizer.eos_token_id:
#             break

#         generated_ids.append(predicted_id.item()) # Append the predicted ID as an integer
#         output = torch.cat([output, predicted_id], dim=-1)  # Prepare for the next iteration


#     summary = tokenizer.decode(generated_ids, skip_special_tokens=True)
#     return summary


# def summarize(input_document, model, tokenizer, max_length=512, decoder_maxlen=100):
#     return evaluate(model=model, input_text=input_document, tokenizer=tokenizer, max_length=max_length, decoder_maxlen=decoder_maxlen)




In [ ]:
# example_text = '''
# Ralph Mata was an internal affairs lieutenant for the Miami-Dade Police Department, working in the division that investigates allegations of wrongdoing by cops.

# Outside the office, authorities allege that the 45-year-old longtime officer worked with a drug trafficking organization to help plan a murder plot and get guns.

# A criminal complaint unsealed in U.S. District Court in New Jersey Tuesday accuses Mata, also known as “The Milk Man,” of using his role as a police officer to help the drug trafficking organization in exchange for money and gifts, including a Rolex watch.
#     '''



# print(summarize( example_text,transformer,tokenizer,decoder_maxlen=75)) # Example usage

uchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuchuch
